# Part 1

In [1]:
import pyparsing as pp
import numpy as np
from functools import partial

In [2]:
uint = pp.Word(pp.nums).set_parse_action(lambda toks: int(toks[0]))
monkey = pp.Suppress("Monkey") + uint("monkey_id") + pp.Suppress(":")

starting_items = pp.Suppress("Starting items:") + pp.delimited_list(uint)

def operation_func(string, location, tokens):
    """Unused, lambdas being unfortunately much slower"""
    if tokens.operator == op_add:
        return lambda x : (x if tokens[0] == old else tokens[0]) + (x if tokens[2] == old else tokens[2])
    elif tokens.operator == op_mult:
        return lambda x : (x if tokens[0] == old else tokens[0]) * (x if tokens[2] == old else tokens[2])
    else:
        raise ValueError("Unsupported operation")

def operation_func(string, location, tokens):
    if not tokens[0] == old:
        raise ValueError("Unsupported left operand")
    if tokens.operator == op_add:
        if tokens[2] == old:
            return partial(int.__mul__, 2)
        else:
            return partial(int.__add__, tokens[2])
    elif tokens.operator == op_mult:
        if tokens[2] == old:
            return partial(pow, exp=2)
        else:
            return partial(int.__mul__, tokens[2])
    else:
        raise ValueError("Unsupported operation")

op_add = pp.Literal("+")
op_mult = pp.Literal("*")
op = op_add | op_mult
old = pp.Literal("old")
variable = old | uint
operation = pp.Suppress("Operation: new =") + variable + op("operator") + variable
operation.setParseAction(operation_func)

test_true = pp.Suppress("If true: throw to monkey") + uint("true_target")
test_false = pp.Suppress("If false: throw to monkey") + uint("false_target")
test_result = test_true + test_false | test_false + test_true
test = pp.Suppress("Test: divisible by") + uint("divisor") + test_result

expr = monkey + starting_items("item_list") + operation("operation") + test

In [3]:
filename = "inputs/12-11.txt"
with open(filename, "r") as f:
    data = f.read()

In [4]:
def init_monkeys(data):
    monkeys = {}
    for r, *_ in expr.scan_string(data):
        monkeys[r.monkey_id] = {
            "items": r.item_list.as_list(),
            "operation": r.operation,
            "divisor": r.divisor,
            "true_target": r.true_target,
            "false_target": r.false_target,
            "inspect": 0
        }
    return monkeys

In [5]:
monkeys = init_monkeys(data)

for _ in range(20):
    for monkey in monkeys.values():
        for item in monkey["items"]:
            monkey["inspect"] += 1
            item = monkey["operation"](item)
            item = item // 3
            if item % monkey["divisor"] == 0:
                target = monkey["true_target"]
            else:
                target = monkey["false_target"]
            monkeys[target]["items"].append(item)
        monkey["items"] = []

print(f"Monkey business is {np.product(sorted([monkey['inspect'] for monkey in monkeys.values()])[-2:], dtype=np.int64)}")

Monkey business is 78678


# Part 2

In [6]:
monkeys = init_monkeys(data)
lcm = int(np.lcm.reduce([monkey["divisor"] for monkey in monkeys.values()]))

for _ in range(10000):
    for monkey in monkeys.values():
        for item in monkey["items"]:
            monkey["inspect"] += 1
            item = item % lcm
            item = monkey["operation"](item)
            if item % monkey["divisor"] == 0:
                target = monkey["true_target"]
            else:
                target = monkey["false_target"]
            monkeys[target]["items"].append(item)
        monkey["items"] = []

print(f"Monkey business is {np.product(sorted([monkey['inspect'] for monkey in monkeys.values()])[-2:], dtype=np.int64)}")

Monkey business is 15333249714
